## Import Libraries
In order to generate a bag of words representation we need the following libraries

In [1]:
import numpy as np
import json

## Import Data


In [2]:
data_set = np.array([ [x['topics'], x['citations']] for x in json.load(open('train-1.json')) if x['topics']])
# data_set = np.array([ [ np.array(x['topics']) , x['citations'] ] for x in json.load(open('train-1.json'))[:3] if x['topics']])

<ipython-input-2-4a3b7c51e9cc>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data_set = np.array([ [x['topics'], x['citations']] for x in json.load(open('train-1.json')) if x['topics']])


## Proportion analysis of journals/venues
Aim of the code is to divide the data in n equal portions, where the 'lowest' division accounts for the group of X that contributes the least to a given Y.

In [3]:
def unique_Array_of_nested_array(x):
    topics_array = list(map((lambda z: z[0]), x))
    topics_array = np.concatenate(topics_array)
    return np.unique(topics_array,return_counts=True)

def divide_data_in_portions(XY):
    #split data 
    X, _ = np.split(XY, 2, 1)

    # Get unique topics
    unique_topics = unique_Array_of_nested_array(X)
    unique_topics_lenght = len(unique_topics[0])

    # Generate 0 arrays in order to fill them with the counts
    zero_arrays = np.zeros((3,unique_topics_lenght))

    # Merge arrays together
    merge = [unique_topics[0], unique_topics[1].astype(int), zero_arrays[0].astype(int), zero_arrays[1].astype(int), zero_arrays[2].astype(float)]

    # Compute citations per topic
    for x in XY :
        for topic in x[0]:
            merge[2][np.where(merge[0] == topic)[0][0]] += int(x[1])


    # Compute citations per topic divided by the amount of articles
    for (i, j) in enumerate(merge[3]):
        merge[3][i] = (merge[2][i]/merge[1][i])

    # Compute new summed citations
    summed_citations = np.sum(merge[3])

    result = []
    # Compute percentage of new total sum
    for (i, j) in enumerate(merge[3]):
        merge[4][i] = 100/summed_citations*merge[3][i]
        result.append((merge[0][i],merge[1][i],merge[2][i],merge[3][i],merge[4][i]))

    return result

division = divide_data_in_portions(data_set)

## Plot the line
In order to make the data more acessible, the data can be plotted

In [4]:
%pylab inline --no-import-all 

division = divide_data_in_portions(data_set)
dtype = [('field_of_study', 'S20'), ('count', int), ('summed_citations', int), ('average_citations', float), ('contribution', float)]

# create a structured array
structured_array = np.array(division, dtype=dtype)
sorted = np.sort(structured_array, order='contribution')
print(sorted[-3:])

Populating the interactive namespace from numpy and matplotlib


UnicodeEncodeError: 'ascii' codec can't encode character '\u2013' in position 8: ordinal not in range(128)